# Rede neural convolucional para classificação binária de imagens (treino)

Importação das bibliotecas necessárias

In [1]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import (Dropout, Flatten, Dense, Conv2D, Activation, MaxPooling2D)
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

Using TensorFlow backend.


Definição dos caminhos para as pastas contendo as imagens e o tamanho do batch

In [2]:
train_path = 'dataset1/train'
validation_path = 'dataset1/valid'
test_path = 'dataset1/test'
batch_size = 64

Gerador de imagens para o treino e validação. As imagens do treino sofrem "augmentation" para aumentar a variabilidade de imagens a serem treinadas para o modelo.

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = False)
valid_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(train_path,
                                                   target_size = (150,150),
                                                   batch_size = batch_size,
                                                   class_mode = 'binary')

validation_generator = valid_datagen.flow_from_directory(validation_path,
                                                        target_size = (150,150),
                                                        batch_size = batch_size,
                                                        class_mode = 'binary')

Found 21557 images belonging to 2 classes.
Found 6000 images belonging to 2 classes.


In [5]:
train_generator.class_indices

{'Desparasitado': 0, 'Parasitado': 1}

Definição do modelo padrão para uma CNN (Convolutional neural network).
O modelo consiste nos seguintes passos:

Convolução -> Pooling espacial (subsampling) (3x) -> Flatten (array) -> Dense (conexão total dos neurons) -> Sigmoid


In [33]:
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu')) #tanh
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

In [34]:
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu')) #tanh
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

In [35]:

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu')) #tanh
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

In [36]:
model.add(Flatten())
model.add(Dense(96))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1)) #dual
model.add(Activation('sigmoid'))

Compilação do modelo

In [37]:
model.compile(loss='binary_crossentropy',
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

Treinamento com fit_generator

Callback para parada de treinamento caso não haja decaimento significativo do val_loss por 10 epochs.
Callback para salvamento do modelo a cada epoch. (save_best_only = True, para salvar apenas o melhor)

In [38]:
#model.fit_generator(train_generator, [0.2613402856089203, 0.9037591365833462]
#                   steps_per_epoch = 200,
#                   epochs = 50,
#                   validation_data = validation_generator,
#                   validation_steps = 1000)

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')

mcp_save = ModelCheckpoint('model.{epoch:02d}-{val_loss:.2f}.hdf5', 
                           save_best_only=False,
                           save_weights_only=False,
                           monitor='val_loss', 
                           mode='min')

model.fit_generator(train_generator,
                   steps_per_epoch = 500,
                   epochs = 70,
                   callbacks=[earlyStopping, mcp_save],
                   validation_data = validation_generator,
                   validation_steps = 1000)


Epoch 1/70
500/500 [==============================] - 478s 956ms/step - loss: 0.5843 - acc: 0.6850 - val_loss: 0.2848 - val_acc: 0.9265
Epoch 2/70
500/500 [==============================] - 642s 1s/step - loss: 0.2268 - acc: 0.9259 - val_loss: 0.2261 - val_acc: 0.9315
Epoch 3/70
500/500 [==============================] - 592s 1s/step - loss: 0.2045 - acc: 0.9381 - val_loss: 0.2357 - val_acc: 0.9288
Epoch 4/70
500/500 [==============================] - 674s 1s/step - loss: 0.1983 - acc: 0.9407 - val_loss: 0.2062 - val_acc: 0.9337
Epoch 5/70
500/500 [==============================] - 663s 1s/step - loss: 0.1751 - acc: 0.9476 - val_loss: 0.1976 - val_acc: 0.9340
Epoch 6/70
500/500 [==============================] - 676s 1s/step - loss: 0.1671 - acc: 0.9505 - val_loss: 0.2102 - val_acc: 0.9337
Epoch 7/70
500/500 [==============================] - 581s 1s/step - loss: 0.1633 - acc: 0.9521 - val_loss: 0.1987 - val_acc: 0.9352
Epoch 8/70
500/500 [==============================] - 567s 1s/step